In [1]:
import cv2
import numpy as np
import time

In [2]:
def my_padding(img, shape, boundary = 0):
    '''
    :param img: image for boundary padding
    :param shape: kernel's shape
    :param boundary: default = 0, zero-padding : 0, repetition : 1, mirroring : 2
    :return: padding is done.
    '''
    row, col = len(img), len(img[0])
    pad_sizeY, pad_sizeX = shape[0] // 2, shape[1] //2
    res = np.zeros((row + (2 * pad_sizeY), col + (2 * pad_sizeX)), dtype=np.float)
    pad_row, pad_col = len(res), len(res[0])
    if pad_sizeY == 0 :
        res[pad_sizeY:, pad_sizeX:-pad_sizeX] = img.copy()
    elif pad_sizeX == 0:
        res[pad_sizeY:-pad_sizeY, pad_sizeX:] = img.copy()
    else:
        res[pad_sizeY:-pad_sizeY, pad_sizeX:-pad_sizeX] = img.copy()
    if boundary == 0:
        return res
    elif boundary == 1:
        res[0:pad_sizeY, 0:pad_sizeX] = img[0, 0]  # Top-Left
        res[-pad_sizeY:, 0:pad_sizeX] = img[row - 1, 0]  # Bottom-Left
        res[0:pad_sizeY, -pad_sizeX:] = img[0, col - 1]  # Top-Right
        res[-pad_sizeY:, -pad_sizeX:] = img[row - 1, col - 1]  # Bottom-Right
        # axis = 1, 열반복, axis = 0, 행반복. default 0
        res[0:pad_sizeY, pad_sizeX:pad_col-pad_sizeX] = np.repeat(img[0:1, 0:], [pad_sizeY], axis=0)  # Top
        res[pad_row-pad_sizeY:, pad_sizeX:pad_col-pad_sizeX] = np.repeat(img[row - 1:row, 0:], [pad_sizeY], axis=0)  # Bottom
        res[pad_sizeY:pad_row-pad_sizeY, 0:pad_sizeX] = np.repeat(img[0:, 0:1], [pad_sizeX], axis=1)  # Left
        res[pad_sizeY:pad_row-pad_sizeY, pad_col-pad_sizeX:] = np.repeat(img[0:, col - 1:col], [pad_sizeX], axis=1)  # Right
        return res
    elif boundary == 2:
        res[0:pad_sizeY, 0:pad_sizeX] = np.flip(img[0:pad_sizeY, 0:pad_sizeX])  # Top-Left
        res[-pad_sizeY:, 0:pad_sizeX] = np.flip(img[-pad_sizeY:, 0:pad_sizeX])  # Bottom-Left
        res[0:pad_sizeY, -pad_sizeX:] = np.flip(img[0:pad_sizeY, -pad_sizeX:])  # Top-Right
        res[-pad_sizeY:, -pad_sizeX:] = np.flip(img[-pad_sizeY:, -pad_sizeX:])  # Bottom-Right

        res[pad_sizeY:pad_row-pad_sizeY, 0:pad_sizeX] = np.flip(img[0:, 0:pad_sizeX], 1)  # Left
        res[pad_sizeY:pad_row-pad_sizeY, pad_col-pad_sizeX:] = np.flip(img[0:, col-pad_sizeX:], 1)  # Right
        res[0:pad_sizeY, pad_sizeX:pad_col-pad_sizeX] = np.flip(img[0:pad_sizeY, 0:], 0)  # Top
        res[pad_row-pad_sizeY:, pad_sizeX:pad_col-pad_sizeX] = np.flip(img[row-pad_sizeY:, 0:], 0)  # Bottom
        
        return res


In [3]:
def my_gaussian(shape, sigma):
    '''
    :param shape: shape of the Gaussian kernel you want to create, enter it in the form (5, 5) or (1, 5)
    :param sigma: standard deviation for Gaussian distribution
    :return: Gaussian kernel of shape
    '''

    m, n = shape[0] // 2, shape[1] // 2
    y, x = np.ogrid[-m: m+1, -n: n+1]
    # Coefficients disappear in the course of normalization.
    gaus = np.exp(-(x*x + y*y) / (2.*sigma*sigma)) # When inverted, X and Y are the same
    
    gaus_sum = gaus.sum()
    if gaus_sum != 0:
        gaus /= gaus_sum
    
    return gaus

In [4]:
def gaus_filtering(img, kernel, boundary = 0):
    '''
    :param img: Image to apply the Gaussian filter
    :param kernel: Gaussian Kernel to apply the image
    :param boundary: parameter (0 : zero-padding, default, 1: repetition, 2:mirroring) 'bout processing boundary
    :return: gaussian filtering is done using the input kernel.
    '''
    row = len(img)
    col = len(img[0])
    ksizeX = kernel.shape[1]
    ksizeY = kernel.shape[0]
    pad_image = my_padding(img, (ksizeY, ksizeX), boundary=boundary)    # create the image padded.
    filtered_img = np.zeros((row, col), dtype=np.float32) # Due to processing the negetive number and decimal point, declare the float.
    for i in range(row):
        for j in range(col):
            filtered_img[i, j] = np.sum(np.multiply(pad_image[i: i+ksizeY, j: j+ksizeX], kernel))

    return filtered_img

#### Performance Comparison  
This directly identifies the difference between the speed of the first and second-dimensional kernel operations. The output of first line is result of 1-D kernel's operations and next line is result of 2-D kernel's operations. if it's correct, 1-D kernel's operation is faster than the another.  

In [ ]:
src = cv2.imread("./sample/lena.tiff", cv2.IMREAD_GRAYSCALE)
gaus2D = my_gaussian((101, 101), 13)
gaus1D = my_gaussian((1, 101), 13)

start = time.perf_counter()     # time measurement
img2D = gaus_filtering(src, gaus2D, boundary=0)
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
img1D = gaus_filtering(src, gaus1D, boundary=0)
img1D = gaus_filtering(img1D, gaus1D.T, boundary=0)
end = time.perf_counter()
print(end-start)

cv2.imshow('img1D', img1D.astype(np.uint8))
cv2.imshow('img2D', img2D.astype(np.uint8))
cv2.waitKey()
cv2.destroyAllWindows()


5.652229591000001
3.3892775340000014
